- All non-negative coefficients
- Unbounded on the higher side
- Bounds for Intercept depend on no_intercept flag also
- Specific order for some coefficients

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from scipy.optimize import lsq_linear

# User inputs

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
no_intercept = True

In [4]:
target = 'SalePrice'

In [9]:
# Features in the expected ascending order of coefficients
features_ordered = ['PoolArea', 'LotArea', 'GrLivArea']
# Features without any order for coefficients
features_unordered = ['TotalBsmtSF', 'GarageArea']
features = features_ordered + features_unordered
features

['PoolArea', 'LotArea', 'GrLivArea', 'TotalBsmtSF', 'GarageArea']

# Constraints

In [7]:
# Initialize coefficients
len_coeffs = len(features) + 1
coeffs = list(np.zeros(len_coeffs))
print("Initialized coefficients:", coeffs)

Initialized coefficients: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [8]:
# Put constraints 
min_con = list(np.zeros(len_coeffs))
max_con = [np.inf for i in range(len_coeffs)]
if no_intercept:
    max_con[0] = 0.0001
print("Minimum constraints:", min_con)
print("Maximum constraints:", max_con)

Minimum constraints: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Maximum constraints: [0.0001, inf, inf, inf, inf, inf]


# Model

y = X0 + X1*PoolArea + (X1+X2)*LotArea + (X1+X2+X3)*GrLivArea + X4*TotalBsmtSF  + X5*GarageArea

y = X0 + X1*(PoolArea+LotArea+GrLivArea) + X2*(LotArea+GrLivArea) + X3*GrLivArea + X4*TotalBsmtSF + X5*GarageArea

In [10]:
# Feature engineer   
X = df[features].copy()
X['F1'] = X[features[0]] + X[features[1]] + X[features[2]]
X['F2'] = X[features[1]] + X[features[2]]
X['F3'] = X[features[2]]
X['F4'] = X[features[3]]
X['F5'] = X[features[4]]
X = X.drop(features, axis=1)

In [11]:
# Convert independent variables to a matrix
X = X.values

# Add an array of ones to act as intercept coefficient
ones = np.ones(X.shape[0])
# Combine array of ones and indepedent variables
X = np.concatenate((ones[:, np.newaxis], X), axis=1)
X

array([[1.0000e+00, 1.0160e+04, 1.0160e+04, 1.7100e+03, 8.5600e+02,
        5.4800e+02],
       [1.0000e+00, 1.0862e+04, 1.0862e+04, 1.2620e+03, 1.2620e+03,
        4.6000e+02],
       [1.0000e+00, 1.3036e+04, 1.3036e+04, 1.7860e+03, 9.2000e+02,
        6.0800e+02],
       ...,
       [1.0000e+00, 1.1382e+04, 1.1382e+04, 2.3400e+03, 1.1520e+03,
        2.5200e+02],
       [1.0000e+00, 1.0795e+04, 1.0795e+04, 1.0780e+03, 1.0780e+03,
        2.4000e+02],
       [1.0000e+00, 1.1193e+04, 1.1193e+04, 1.2560e+03, 1.2560e+03,
        2.7600e+02]])

In [12]:
# Convert target variable to a matrix
y = df[target].values
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [13]:
# Run optimization
results = lsq_linear(X, y, bounds=(min_con, max_con), lsmr_tol='auto')
print("Results:\n", results)

Results:
  active_mask: array([-1, -1,  0,  0,  0,  0])
        cost: 1591264994306.908
         fun: array([-15570.8800255 ,  -6211.14423445, -17020.4392965 , ...,
       -51622.04361478,  -6813.20555247,   9554.21332259])
     message: 'The relative change of the cost function is less than `tol`.'
         nit: 18
  optimality: 0.000170350316577374
      status: 2
     success: True
           x: array([1.09691706e-40, 1.01535386e-20, 1.81752451e-01, 5.95417512e+01,
       4.27659107e+01, 9.60921551e+01])


In [14]:
if results.success:
    # Transform the coefficients back to the context of original features 
    coeffs[0] = results.x[0]
    coeffs[1] = results.x[1]
    coeffs[2] = results.x[1] + results.x[2]
    coeffs[3] = results.x[1] + results.x[2] + results.x[3]
    coeffs[4] = results.x[4]
    coeffs[5] = results.x[5]
    print("Final Coefficients (including intercept):", coeffs)
else:
    print("Convergence was not achieved!")

Final Coefficients (including intercept): [1.0969170605431398e-40, 1.0153538621511846e-20, 0.18175245111176683, 59.72350363625728, 42.76591070810782, 96.09215507019684]
